In [1]:
# Section 1: Import Libraries
# Purpose: Load all required libraries for building, training, and testing the InceptionV3 model on CIFAR-10.
# We use tensorflow.keras consistently to avoid compatibility issues with TensorFlow 2.19.0.
import tensorflow as tf  # Core TensorFlow library for deep learning operations
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input  # InceptionV3 model and preprocessing function
from tensorflow.keras import models, layers, optimizers  # For building and compiling the model
from tensorflow.keras.utils import to_categorical  # For one-hot encoding labels (replaces np_utils)
from tensorflow.keras.models import load_model  # For saving/loading models
from tensorflow.keras.preprocessing import image  # For image preprocessing in predictions
from tensorflow.keras.datasets import cifar10  # CIFAR-10 dataset (50,000 training, 10,000 test images)
import numpy as np  # For numerical operations (e.g., array manipulation)
import matplotlib.pyplot as plt  # For visualizing images and results
import cv2  # OpenCV for image loading and resizing
from tensorflow.keras.models import Model  # Functional API for custom model building
from tensorflow.keras.layers import UpSampling2D, Flatten, Reshape, Dropout, Dense, Multiply, Dot, Concatenate, Embedding, Conv2D, Input  # Layers for model architecture

In [2]:
# Section 2: Verify TensorFlow Version
# Purpose: Confirm the TensorFlow version to ensure compatibility (should output 2.19.0).
print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.19.0


In [3]:
# Section 3: Initialize InceptionV3 Base Model
# Purpose: Load pre-trained InceptionV3 with ImageNet weights, excluding top layers for custom classification.
# Input shape is 224x224x3, as required by InceptionV3.
conv_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  # Load InceptionV3 without fully connected layers

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [4]:
# Freeze pre-trained layers to use ImageNet weights without retraining (saves time and prevents overfitting)
for layer in conv_base.layers:
    layer.trainable = False

In [5]:
# Section 4: Inspect InceptionV3 Layers
# Purpose: Print layer names and trainability to verify the model structure and confirm layers are frozen.
print("InceptionV3 Layers and Trainability:")
for layer in conv_base.layers:
    print(f"{layer.name}: trainable={layer.trainable}")

InceptionV3 Layers and Trainability:
input_layer: trainable=False
conv2d: trainable=False
batch_normalization: trainable=False
activation: trainable=False
conv2d_1: trainable=False
batch_normalization_1: trainable=False
activation_1: trainable=False
conv2d_2: trainable=False
batch_normalization_2: trainable=False
activation_2: trainable=False
max_pooling2d: trainable=False
conv2d_3: trainable=False
batch_normalization_3: trainable=False
activation_3: trainable=False
conv2d_4: trainable=False
batch_normalization_4: trainable=False
activation_4: trainable=False
max_pooling2d_1: trainable=False
conv2d_8: trainable=False
batch_normalization_8: trainable=False
activation_8: trainable=False
conv2d_6: trainable=False
conv2d_9: trainable=False
batch_normalization_6: trainable=False
batch_normalization_9: trainable=False
activation_6: trainable=False
activation_9: trainable=False
average_pooling2d: trainable=False
conv2d_5: trainable=False
conv2d_7: trainable=False
conv2d_10: trainable=False
co

In [6]:
# Section 5: Display Model Summary
# Purpose: Show the architecture of InceptionV3 (e.g., layer types, output shapes, parameters).
conv_base.summary()

Model: "inception_v3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 21,802,784 (83.17 MB)

In [7]:
# Section 6: Load and Preprocess CIFAR-10 Dataset
# Purpose: Load CIFAR-10 data, normalize images to [0, 1], and one-hot encode labels for 10 classes.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()  # Load CIFAR-10: 50,000 train, 10,000 test images (32x32x3)
x_train = x_train / 255.0  # Normalize pixel values to [0, 1] for better neural network performance
x_test = x_test / 255.0    # Normalize test images similarly
y_train = to_categorical(y_train, 10)  # One-hot encode labels (e.g., 3 -> [0,0,0,1,0,0,0,0,0,0])
y_test = to_categorical(y_test, 10)    # Same for test labels
print(f"Training data shape: {x_train.shape}")  # Expected: (50000, 32, 32, 3)
print(f"Test data shape: {x_test.shape}")      # Expected: (10000, 32, 32, 3)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Training data shape: (50000, 32, 32, 3)
Test data shape: (10000, 32, 32, 3)


In [8]:
# Section 7: Build the Model
# Purpose: Create a sequential model that upsamples CIFAR-10 images and uses InceptionV3 for feature extraction.
model = models.Sequential()  # Initialize a sequential model
model.add(layers.Input(shape=(32, 32, 3)))  # Define input shape for CIFAR-10 images
model.add(UpSampling2D(size=(7, 7)))  # Upsample 32x32 to 224x224 (32*7=224) for InceptionV3
model.add(conv_base)  # Add pre-trained InceptionV3 as a feature extractor
model.add(layers.Flatten())  # Flatten the output to a 1D vector
model.add(layers.Dense(128, activation='relu'))  # Dense layer for feature processing
model.add(layers.Dropout(0.2))  # Dropout to prevent overfitting
model.add(layers.Dense(10, activation='softmax'))  # Output layer for 10 CIFAR-10 classes

In [9]:
# Section 8: Compile the Model
# Purpose: Configure the model with optimizer, loss function, and metrics for training.
model.compile(
    optimizer=optimizers.RMSprop(learning_rate=0.0001),  # RMSprop with low learning rate for fine-tuning
    loss='categorical_crossentropy',  # Loss function for multi-class classification
    metrics=['acc']  # Track accuracy during training
)

In [ ]:
# Section 9: Train the Model
# Purpose: Train the model on CIFAR-10 data to learn weights for the custom layers.
# We use 5 epochs for better learning (1 epoch was too short).
history = model.fit(
    x_train, y_train,  # Training data and labels
    epochs=5,  # Number of passes through the dataset (adjustable)
    batch_size=64,  # Number of samples per batch
    validation_data=(x_test, y_test),  # Evaluate on test data
    verbose=1  # Show training progress
)

Epoch 1/5
 52/782 ━━━━━━━━━━━━━━━━━━━━ 1:38:29 8s/step - acc: 0.3592 - loss: 3.3435

In [ ]:
# Section 10: Save the Trained Model
# Purpose: Save the trained model to a file for later use.
model.save("cifar_model.h5")  # Save as HDF5 file
print("Model saved as cifar_model.h5")

In [ ]:
# Section 11: Load the Saved Model
# Purpose: Reload the model to verify it saved correctly and for predictions.
model = load_model("cifar_model.h5")  # Load the saved model
print(f"Loaded model name: {model.name}")

In [ ]:
# Section 12: Prediction with CIFAR-10 Test Image
# Purpose: Test the model on a CIFAR-10 test image to avoid URL/file issues.
# We use a 32x32 image directly, letting the model's UpSampling2D layer resize to 224x224.
test_image = x_test[0]  # Select first test image (shape: 32x32x3)
test_image = test_image * 255.0  # Reverse normalization (from [0, 1] to [0, 255] for visualization)
assert test_image.shape == (32, 32, 3), f"Expected shape (32, 32, 3), got {test_image.shape}"  # Verify input shape
x = np.expand_dims(test_image, axis=0)  # Add batch dimension (shape: 1x32x32x3)
x = preprocess_input(x)  # Apply InceptionV3 preprocessing (scales pixels, e.g., to [-1, 1])
preds = model.predict(x)  # Predict class probabilities
class_idx = np.argmax(preds[0])  # Get the predicted class index
print("Prediction probabilities (CIFAR-10 test image):", preds[0])
print("Predicted class index:", class_idx)
# Visualize the test image (before resizing, to show original 32x32 image)
plt.imshow(test_image.astype(np.uint8))  # Cast to uint8 for correct rendering
plt.title(f"Predicted class: {class_idx} (CIFAR-10 classes: 0=airplane, 1=automobile, 2=bird, 3=cat, 4=deer, 5=dog, 6=frog, 7=horse, 8=ship, 9=truck)")
plt.show()

In [ ]:
# Section 13: Prediction with External Image (Optional)
# Purpose: Predict on an external image with error checking to avoid OpenCV errors.
# Uncomment this section and replace 'car.jpg' with your image path to use it.

image_path = '/content/img3.jpeg'  # Replace with your actual image path (e.g., './car.jpg')
input_img = cv2.imread(image_path)  # Load the image using OpenCV
if input_img is None:
    raise ValueError(f"Image not found at {image_path}")  # Check for invalid path
input_img = cv2.resize(input_img, dsize=(32, 32), interpolation=cv2.INTER_CUBIC)  # Resize to 32x32 to match model input
x = image.img_to_array(input_img)  # Convert to NumPy array
x = np.expand_dims(x, axis=0)  # Add batch dimension
x = preprocess_input(x)  # Apply InceptionV3 preprocessing
preds = model.predict(x)  # Predict class probabilities
class_idx = np.argmax(preds[0])  # Get the predicted class index
print("Prediction probabilities (external image):", preds[0])
print("Predicted class index:", class_idx)
plt.imshow(input_img)  # Display the image
plt.title(f"Predicted class: {class_idx} (CIFAR-10 classes: 0=airplane, 1=automobile, 2=bird, 3=cat, 4=deer, 5=dog, 6=frog, 7=horse, 8=ship, 9=truck)")
plt.show()